In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import sklearn.preprocessing as preprocessing
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FormatStrFormatter

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/preprocesssed/xtrain.csv
/kaggle/input/preprocesssed/xtest.csv
/kaggle/input/preprocesssed/ytrain.csv


In [3]:
xtrain=pd.read_csv("/kaggle/input/preprocesssed/xtrain.csv")
ytrain=pd.read_csv("/kaggle/input/preprocesssed/ytrain.csv")
test=pd.read_csv("/kaggle/input/preprocesssed/xtest.csv")

In [4]:
import xgboost as xg

In [ ]:
# xgb = xgboost.XGBClassifier(learning_rate=0.5,max_depth=5,n_estimators=5000,subsample=0.5,colsample_bytree=0.5,eval_metric='auc',verbosity=1)

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_valid, Y_train, Y_valid = train_test_split(xtrain, ytrain, test_size=0.2)

In [8]:
eval_set = [(X_valid, Y_valid)]

In [ ]:
xgb.fit(xtrain,ytrain,early_stopping_rounds=1,eval_set=eval_set,verbose=True)

In [ ]:
ypred=xgb.predict(test)

In [ ]:
test["isFraud"]=ypred
ypred=test["isFraud"]
ypred.to_csv("f(xgbTestboostViva).csv")
test=test.drop(['isFraud'],axis=1,inplace=False)

In [22]:
params={
 "learning_rate"    : [0.01, 0.03 , 0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ,0.40 ,0.50  ] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
    
}

In [23]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [28]:
classifier=xg.XGBClassifier()

In [29]:
random_search=RandomizedSearchCV(classifier,param_distributions=params,n_iter=5,scoring='roc_auc',n_jobs=-1,cv=5,verbose=3)

In [30]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [31]:
from datetime import datetime
# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(xtrain,ytrain)
timer(start_time) # timing ends here for "start_time" variable

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 2/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.25, max_depth=15, min_child_weight=1;, score=1.000 total time=31.0min
[CV 1/5] END colsample_bytree=0.5, gamma=0.4, learning_rate=0.2, max_depth=10, min_child_weight=5;, score=0.980 total time=31.8min
[CV 1/5] END colsample_bytree=0.3, gamma=0.1, learning_rate=0.03, max_depth=8, min_child_weight=5;, score=0.976 total time=17.4min
[CV 4/5] END colsample_bytree=0.3, gamma=0.1, learning_rate=0.03, max_depth=8, min_child_weight=5;, score=0.999 total time=16.9min
[CV 3/5] END colsample_bytree=0.4, gamma=0.1, learning_rate=0.3, max_depth=4, min_child_weight=3;, score=1.000 total time=11.2min
[CV 2/5] END colsample_bytree=0.5, gamma=0.3, learning_rate=0.05, max_depth=10, min_child_weight=1;, score=1.000 total time=31.4min
[CV 4/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.25, max_depth=15, min_child_weight=1;, score=1.000 total time=31.0min
[CV 2/5] END colsampl

In [32]:
random_search.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.3,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0.2, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.25, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=15, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [5]:
from sklearn.model_selection import train_test_split

In [25]:
X_train, X_valid, Y_train, Y_valid = train_test_split(xtrain, ytrain, test_size=0.4)

In [26]:
eval_set = [(X_valid, Y_valid)]

In [27]:
xgb = xg.XGBClassifier(base_score=0.5,colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.3, enable_categorical=False, gamma=0.2, gpu_id=-1, grow_policy='depthwise', interaction_constraints='',
              learning_rate=0.25, max_bin=256
             ,max_depth=15, max_leaves=0, min_child_weight=1, monotone_constraints='()', n_estimators=10000,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1,eval_metric=['auc','logloss'],verbosity=1)

In [31]:
xgb.fit(X_train,Y_train,early_stopping_rounds=100,eval_set=eval_set,verbose=True)

[0]	validation_0-auc:0.98259	validation_0-logloss:0.51241
[1]	validation_0-auc:0.99317	validation_0-logloss:0.38763
[2]	validation_0-auc:0.99440	validation_0-logloss:0.30373
[3]	validation_0-auc:0.99499	validation_0-logloss:0.24239
[4]	validation_0-auc:0.99541	validation_0-logloss:0.19893
[5]	validation_0-auc:0.99582	validation_0-logloss:0.16536
[6]	validation_0-auc:0.99608	validation_0-logloss:0.13763
[7]	validation_0-auc:0.99626	validation_0-logloss:0.11737
[8]	validation_0-auc:0.99630	validation_0-logloss:0.10174
[9]	validation_0-auc:0.99630	validation_0-logloss:0.09115
[10]	validation_0-auc:0.99643	validation_0-logloss:0.08039
[11]	validation_0-auc:0.99651	validation_0-logloss:0.07307
[12]	validation_0-auc:0.99666	validation_0-logloss:0.06652
[13]	validation_0-auc:0.99677	validation_0-logloss:0.06109
[14]	validation_0-auc:0.99690	validation_0-logloss:0.05696
[15]	validation_0-auc:0.99700	validation_0-logloss:0.05369
[16]	validation_0-auc:0.99712	validation_0-logloss:0.05074
[17]	va

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.3,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=['auc', 'logloss'], gamma=0.2, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.25, max_bin=256,
              max_cat_to_onehot=4, max_delta_step=0, max_depth=15, max_leaves=0,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=10000, n_jobs=0, num_parallel_tree=1,
              predictor='auto', random_state=0, reg_alpha=0, reg_lambda=1, ...)

In [32]:
ypred=xgb.predict(test)

In [33]:
test["isFraud"]=ypred
ypred=test["isFraud"]
ypred.to_csv("f(xgboost270).csv")
test=test.drop(['isFraud'],axis=1,inplace=False)